In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/home/monster/Downloads/kaggle/AV-L&T-loan/train.csv')
test_df = pd.read_csv('/home/monster/Downloads/kaggle/AV-L&T-loan/test.csv')
sub_df = pd.read_csv('/home/monster/Downloads/kaggle/AV-L&T-loan/sample_submission.csv')

In [ ]:
train_df.shape,test_df.shape

In [ ]:
train_df['is_train'] = ''
test_df['is_train'] = ''
data = pd.DataFrame()
train_df.loc[:,'is_train'] = 1
test_df.loc[:,'is_train'] = 0
data = data.append(train_df,sort=False).append(test_df,sort=False)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
#Treat missing values
#86 % values are no so it is safe to impute NO
data['Employment.Type'].fillna('Self employed',inplace=True)

In [ ]:
data.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.shape

In [ ]:
len(data['Employment.Type'].value_counts())

In [ ]:
len(data['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts())

In [ ]:
data['PERFORM_CNS.SCORE.DESCRIPTION'] = data['PERFORM_CNS.SCORE.DESCRIPTION'].replace(' ', '_', regex=True)

In [ ]:
#treat categorical variables
from sklearn.preprocessing import OneHotEncoder

d = pd.DataFrame()

var_mod = ['Employment.Type','PERFORM_CNS.SCORE.DESCRIPTION']

x = pd.get_dummies(data[var_mod])

data  = data.drop(var_mod,axis=1)
data = pd.concat([data,x], axis=1)
    
# for i in var_mod:
#     print(i)
# #     d[str(i)] 
#     dd = le.fit_transform(data.loc[:,i]).toarray()
#     print(dd.shape)
# d.dtypes


In [ ]:
data.shape

In [ ]:
data.describe()['PRIMARY.INSTAL.AMT']

In [ ]:
data['disbursed_amount'].hist(bins=50)
plt.show()

In [ ]:
np.log(data['disbursed_amount']).hist(bins=50)
plt.show()

In [ ]:
data['log_disbursed_amount'] = np.log(data['disbursed_amount'])
data['log_asset_cost'] = np.log(data['asset_cost'])

In [ ]:
type(data['Date.of.Birth'][0])

In [ ]:
type(data['DisbursalDate'][0])

In [ ]:
data['DisbursalDate'][0]

In [ ]:
data['DisbursalDate_month'] = data['DisbursalDate'].map(lambda x : int(x.split('-')[1]))
data['DisbursalDate_dayofmonth'] = data['DisbursalDate'].map(lambda x : int(x.split('-')[0]))

In [ ]:
type(data['AVERAGE.ACCT.AGE'][0])

In [ ]:
type(data['CREDIT.HISTORY.LENGTH'][0])

In [ ]:
train_df = data.loc[data['is_train']==1,:]
train_df = train_df.drop(['is_train'],axis=1)

In [ ]:
test_df = data.loc[data['is_train']==0,:]
test_df = test_df.drop(['is_train','loan_default'],axis=1)

In [ ]:
train_df.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
test_df.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
train_df.shape,test_df.shape

In [ ]:
train_df.columns

In [ ]:

predictors = ['disbursed_amount', 'asset_cost', 'ltv',
       'branch_id', 'supplier_id', 'manufacturer_id', 'Current_pincode_ID',
       'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
       'Employment.Type_Salaried', 'Employment.Type_Self employed',
       'PERFORM_CNS.SCORE.DESCRIPTION_A-Very_Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_B-Very_Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_C-Very_Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_D-Very_Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_E-Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_F-Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_G-Low_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_H-Medium_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_I-Medium_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_J-High_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_K-High_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_L-Very_High_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_M-Very_High_Risk',
       'PERFORM_CNS.SCORE.DESCRIPTION_No_Bureau_History_Available',
       'PERFORM_CNS.SCORE.DESCRIPTION_Not_Scored:_More_than_50_active_Accounts_found',
       'PERFORM_CNS.SCORE.DESCRIPTION_Not_Scored:_No_Activity_seen_on_the_customer_(Inactive)',
       'PERFORM_CNS.SCORE.DESCRIPTION_Not_Scored:_No_Updates_available_in_last_36_months',
       'PERFORM_CNS.SCORE.DESCRIPTION_Not_Scored:_Not_Enough_Info_available_on_the_customer',
       'PERFORM_CNS.SCORE.DESCRIPTION_Not_Scored:_Only_a_Guarantor',
       'PERFORM_CNS.SCORE.DESCRIPTION_Not_Scored:_Sufficient_History_Not_Available','DisbursalDate_month',
       'DisbursalDate_dayofmonth']

outcome = ['loan_default']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(train_df[predictors], train_df[outcome], test_size=0.3, random_state=1234)

In [ ]:
X_train.shape,X_validation.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
alg = RandomForestClassifier()


In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn import metrics 
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

In [ ]:
#kf = loo.split(X_train_std[predictor_var])
kf = KFold(n_splits=5).split(X_train['disbursed_amount'])
error = []
# yavg = pd.DataFrame()

for train, test in kf:
    # Filter training data
    print("training stared")
    train_predictors = X_train[predictors].iloc[train,:]

    # The target we're using to train the algorithm.
    train_target = y_train.iloc[train].values

    # Training the algorithm using the predictors and target.
    alg.fit(train_predictors, train_target)
    y1 = alg.predict(X_train[predictors].iloc[test,:])
#     yavg = yavg.append(pd.Series(y1),ignore_index=True)

    #Record error from each cross-validation run
    error.append(metrics.roc_auc_score(y_train.iloc[test], y1))
    print("training finished")
print ("Cross-Validation AUC Score : %f" % np.mean(error))


In [ ]:
# metrics.roc_auc_score(y_train.iloc[test], )
# np.transpose(yavg.loc[yavg['index']=='mean',:])[1:]
y_train.shape

In [ ]:
def modelfit(alg, dtrain,ytrain,dtest,ytest,test,sub_df, predictors,outcome_var,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=ytrain.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds,verbose_eval=True,
            show_stdv=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], ytrain,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtest[predictors])
    dtrain_predprob = alg.predict_proba(dtest[predictors])[:,1]
    
    dtrain_predprob1 = alg.predict_proba(test[predictors])[:,1]
    sub_df['loan_default']= dtrain_predprob1
    sub_df.to_csv('sub1.csv',index=False)    
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(ytest.values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(ytest, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')  
  

In [ ]:
modelfit(XGBClassifier(), X_train,y_train,X_validation,y_validation,test_df,sub_df,predictors,outcome,useTrainCV=True, cv_folds=5, early_stopping_rounds=50)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
model2 = SVR(kernel = 'rbf')

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.tree import DecisionTreeRegressor
model3 = DecisionTreeRegressor(random_state = 50)

from sklearn.ensemble import RandomForestRegressor
model4 = RandomForestRegressor(n_estimators = 270, max_depth=5,
                               min_samples_split=30,min_samples_leaf=12,max_leaf_nodes=30,min_weight_fraction_leaf=0.1,random_state = 50)

from xgboost import XGBRegressor
model5 = XGBRegressor()

from sklearn.neighbors import KNeighborsRegressor
model6 = KNeighborsRegressor(n_neighbors =37)

from sklearn.linear_model import Ridge, Lasso ,ElasticNet

model7 = Ridge()

model8 = Lasso(max_iter=1000,alpha=2000)

model9 = RandomForestRegressor()

model10 = ElasticNet()

from vecstack import stacking



In [ ]:
model_fn(model4,final_train[predictor_var],final_train[outcome_var],
         final_test[predictor_var],final_test[outcome_var],predictor_var,
         3,'alg2.csv', False, True, False)

In [ ]:
model1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=1,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [ ]:
modelfit(model1, X_train,y_train,X_validation,y_validation,test_df,sub_df,predictors,outcome,useTrainCV=True, cv_folds=5, early_stopping_rounds=50)

In [ ]:
from sklearn.metrics import roc_curve, auc
def Performance(Model,Y,X):
    # Perforamnce of the model
    fpr, tpr, _ = roc_curve(Y, Model.predict_proba(X)[:,1])
    AUC  = auc(fpr, tpr)
    print ('the AUC is : %0.4f' %  AUC)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % AUC)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
Performance(model1,y_validation,X_validation)

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
np.linspace(0,1,11)

In [ ]:
from sklearn.model_selection import cross_val_score
def acc_model(params):
    clf = XGBClassifier(**params)
    return cross_val_score(clf, X_train, y_train).mean()

param_space = {
    'learning_rate': hp.choice('learning_rate', np.linspace(0,1,11)),
    'min_child_weight': hp.choice('min_child_weight', range(1,7)),
    'gamma': hp.choice('gamma', np.linspace(0,1,11)),
    'subsample': hp.choice('subsample', np.linspace(0,1,11)),
    'colsample_bytree': hp.choice('colsample_bytree', np.linspace(0,1,11)),
    'max_depth': hp.choice('max_depth', range(1,20)),
    'max_features': hp.choice('max_features', range(1,55)),
    'n_estimators': hp.choice('n_estimators', range(100,500)),
    'criterion': hp.choice('criterion', ["gini", "entropy"])}

best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=5, trials=trials)
print ('best:')
print (best)


In [ ]:
model1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=300,
 max_depth=8,
 min_child_weight=6,
 gamma=0.6,
 subsample=0.8,
 colsample_bytree=0.2,
 objective= 'binary:logistic',
 nthread=-1,
 seed=27)

modelfit(model1, X_train,y_train,X_validation,y_validation,test_df,sub_df,predictors,outcome,useTrainCV=True, cv_folds=5, early_stopping_rounds=50)